Connected to garfield (Python 3.12.2)

In [1]:
# learners from interacting with me
# predictors that ineracts with the learners. For every pretrained predictors:
# - define an uncertainity prediction 
# - define calling tool / imported model

import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F
import numpy as np 

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [2]:
import os 

In [8]:
from playbook import DataBasket

In [9]:
data = DataBasket()

In [12]:
data._train[0]

{'input': [[0, 0, 5], [0, 5, 0], [5, 0, 0]],
 'output': [[3, 3, 3], [4, 4, 4], [2, 2, 2]]}

In [13]:
data._train[:4]

[{'input': [[0, 0, 5], [0, 5, 0], [5, 0, 0]],
  'output': [[3, 3, 3], [4, 4, 4], [2, 2, 2]]},
 {'input': [[0, 0, 5], [0, 0, 5], [0, 0, 5]],
  'output': [[3, 3, 3], [3, 3, 3], [3, 3, 3]]},
 {'input': [[5, 0, 0], [0, 5, 0], [5, 0, 0]],
  'output': [[2, 2, 2], [4, 4, 4], [2, 2, 2]]},
 {'input': [[0, 5, 0], [0, 0, 5], [0, 5, 0]],
  'output': [[4, 4, 4], [3, 3, 3], [4, 4, 4]]}]

In [14]:
data.transform(x_arrs=data._train[0]['input'])

AttributeError: 'list' object has no attribute 'shape'

In [16]:
sample = torch.tensor(data._train[0]['input'], device=device, requires_grad=True, dtype=torch.float)

In [17]:
sample

tensor([[0., 0., 5.],
        [0., 5., 0.],
        [5., 0., 0.]], device='mps:0', requires_grad=True)

In [19]:
sample.flatten()

tensor([0., 0., 5., 0., 5., 0., 5., 0., 0.], device='mps:0',
       grad_fn=<ViewBackward0>)

In [20]:
sample.flatten().shape

torch.Size([9])

In [21]:
train_data = data._train

In [26]:
all_flatten_shape = set([torch.tensor(i['input'], dtype=torch.float, device=device).flatten().shape for i in train_data])

In [31]:
max_shape = max(all_flatten_shape)[0]
max_shape


900

In [32]:
max_grid_size = 30 # for x and y 
x_size_embedding = nn.Embedding(embedding_dim=max_grid_size, padding_idx=1, device=device)


TypeError: Embedding.__init__() missing 1 required positional argument: 'num_embeddings'

In [33]:
max_grid_size = 30 # for x and y 
x_embedding = nn.Embedding(num_embeddings=max_grid_size, embedding_dim=max_shape, padding_idx=1, device=device)
y_embedding = nn.Embedding(num_embeddings=max_grid_size, embedding_dim=max_shape, padding_idx=1, device=device)



In [35]:
x_embedding.weight.shape

torch.Size([30, 900])

In [36]:
import numpy as np

In [37]:
np.zeros(max_grid_size)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [38]:
sample_grid = np.zeros(max_grid_size)

In [39]:
sample_grid[0] = 1

In [40]:
sample_grid.shape

(30,)

In [46]:
x_embedding(torch.tensor(sample_grid, dtype=torch.int, device=device).unsqueeze(0))

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         ...,
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226]]],
       device='mps:0', grad_fn=<EmbeddingBackward0>)

In [51]:
embed = x_embedding(torch.tensor(sample_grid, dtype=torch.int, device=device).unsqueeze(0))

In [52]:
embed

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         ...,
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226]]],
       device='mps:0', grad_fn=<EmbeddingBackward0>)

In [53]:
embed.shape

torch.Size([1, 30, 900])

In [54]:
torch.concat(embed)

TypeError: concat(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [55]:
torch.vstack(embed, embed)

TypeError: vstack() takes 1 positional argument but 2 were given

In [56]:
torch.vstack([embed, embed])

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         ...,
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         ...,
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226],
         [ 0.7265, -0.8913, -1.1414,  ..., -2.1958,  1.0454, -1.1226]]],
       device='mps:0', grad_fn=<CatBackward0>)

In [57]:
torch.vstack([embed, embed]).shape

torch.Size([2, 30, 900])

In [60]:
torch.sum(embed * embed, dim=0)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.5278, 0.7945, 1.3028,  ..., 4.8215, 1.0929, 1.2602],
        [0.5278, 0.7945, 1.3028,  ..., 4.8215, 1.0929, 1.2602],
        ...,
        [0.5278, 0.7945, 1.3028,  ..., 4.8215, 1.0929, 1.2602],
        [0.5278, 0.7945, 1.3028,  ..., 4.8215, 1.0929, 1.2602],
        [0.5278, 0.7945, 1.3028,  ..., 4.8215, 1.0929, 1.2602]],
       device='mps:0', grad_fn=<SumBackward1>)

In [61]:
torch.sum(embed * embed, dim=0).shape, torch.sum(embed * embed, dim=1).shape

(torch.Size([30, 900]), torch.Size([1, 900]))

In [62]:
torch.sum(embed * embed, dim=1)

tensor([[1.5307e+01, 2.3040e+01, 3.7781e+01, 2.5043e+00, 4.2005e-02, 2.3110e+01,
         5.5526e+00, 1.8952e+01, 1.9702e+01, 1.3703e+02, 2.1788e+01, 1.8760e+01,
         6.1309e+01, 7.4935e+00, 1.0528e+01, 5.3323e+01, 3.6621e+02, 3.4205e+00,
         3.0690e+01, 1.1196e+01, 3.9782e-02, 1.0957e+01, 1.5313e-02, 1.6772e+01,
         6.7608e-01, 4.3334e-01, 7.0157e+01, 7.8361e+01, 8.6918e+00, 6.4098e-01,
         1.1055e+00, 4.1080e-01, 7.7926e+01, 3.0869e+01, 2.7739e-01, 1.5546e+01,
         1.8316e+00, 8.3252e+00, 3.6087e+00, 2.2048e+01, 2.2729e+01, 1.0586e+01,
         3.9875e+00, 1.8028e+00, 1.6959e+01, 4.1169e+01, 1.4785e+01, 4.0899e+01,
         1.1424e-01, 2.1697e+01, 8.9582e+00, 3.0397e+00, 4.0092e-01, 8.5779e-01,
         2.7631e+01, 9.6801e+01, 2.1343e+01, 1.9711e+01, 3.3820e+00, 6.0216e+00,
         4.7612e+01, 4.0938e-02, 4.1248e+00, 1.4884e+00, 3.1421e+00, 7.4105e+01,
         8.2241e+00, 5.6911e+01, 1.0365e+01, 9.2351e+01, 8.0424e+01, 3.0515e-02,
         1.3725e+02, 3.7150e

In [63]:
torch.sum(embed * embed, dim=-1).shape

torch.Size([1, 30])

In [65]:
embed.shape

torch.Size([1, 30, 900])

In [66]:
max_grid_size = 30 # for x and y 
x_embedding = nn.Embedding(num_embeddings=max_grid_size, embedding_dim=max_grid_size, padding_idx=1, device=device)
y_embedding = nn.Embedding(num_embeddings=max_grid_size, embedding_dim=max_grid_size, padding_idx=1, device=device)



In [73]:
embed2 = x_embedding(torch.tensor(sample_grid, dtype=torch.int, device=device).unsqueeze(0))

In [74]:
torch.sum(embed2 * embed2, dim=0)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [8.9257e-01, 6.3402e-01, 1.9678e+00, 2.7466e-01, 4.6836e-03, 1.5847e-02,
         1.5331e-02, 6.0557e-01, 6.9611e-01, 7.5780e-02, 3.4795e-02, 2.8965e-02,
         6.0725e+00, 2.7497e-02, 1.1084e-01, 2.0031e+00, 7.4784e-01, 2.6549e+00,
         3.3841e-02, 1.6590e+00, 5.0323e-01, 1.1886e+00, 2.3199e+00, 2.3395e+00,
         4.1650e+00, 1.0014e-01, 1.5482e-01, 1.9063e-01, 9.6168e-04, 3.2201e-01],
        [8.9257e-01, 6.3402e-01, 1.9678e+00, 2.7466e-01, 4.6836e-03, 1.5847e-02,
         1.5331e-02, 6.0557e-01, 6.9611e-01, 7.5780e-02, 3.4795e-02, 2.8965e-02,
         6.0725e+00, 2.749

In [75]:
torch.sum(embed2 * embed2, dim=0).shape, torch.sum(embed2 * embed2, dim=1).shape, torch.sum(embed2 * embed2, dim=-1).shape 

(torch.Size([30, 30]), torch.Size([1, 30]), torch.Size([1, 30]))

In [76]:
torch.sum(embed2 * embed2, dim=0).flatten()

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        8.9257e-01, 6.3402e-01, 1.9678e+00, 2.7466e-01, 4.6836e-03, 1.5847e-02,
        1.5331e-02, 6.0557e-01, 6.9611e-01, 7.5780e-02, 3.4795e-02, 2.8965e-02,
        6.0725e+00, 2.7497e-02, 1.1084e-01, 2.0031e+00, 7.4784e-01, 2.6549e+00,
        3.3841e-02, 1.6590e+00, 5.0323e-01, 1.1886e+00, 2.3199e+00, 2.3395e+00,
        4.1650e+00, 1.0014e-01, 1.5482e-01, 1.9063e-01, 9.6168e-04, 3.2201e-01,
        8.9257e-01, 6.3402e-01, 1.9678e+00, 2.7466e-01, 4.6836e-03, 1.5847e-02,
        1.5331e-02, 6.0557e-01, 6.9611e-01, 7.5780e-02, 3.4795e-02, 2.8965e-02,
        6.0725e+00, 2.7497e-02, 1.1084e-

In [77]:
torch.sum(embed2 * embed2, dim=0).flatten().shape

torch.Size([900])

In [78]:
max_grid_size = 30 # for x and y 
x_embedding = nn.Embedding(num_embeddings=max_grid_size, embedding_dim=max_grid_size, device=device)
y_embedding = nn.Embedding(num_embeddings=max_grid_size, embedding_dim=max_grid_size, device=device)
embed2 = x_embedding(torch.tensor(sample_grid, dtype=torch.int, device=device).unsqueeze(0))
emb_layer = torch.sum(embed2 * embed2, dim=0)



In [81]:
emb_layer.flatten().shape

torch.Size([900])

In [82]:
embed2.shape

torch.Size([1, 30, 30])

In [83]:
emb_layer.shape

torch.Size([30, 30])

In [84]:
sample

tensor([[0., 0., 5.],
        [0., 5., 0.],
        [5., 0., 0.]], device='mps:0', requires_grad=True)

In [85]:
sample.flatten()

tensor([0., 0., 5., 0., 5., 0., 5., 0., 0.], device='mps:0',
       grad_fn=<ViewBackward0>)

In [86]:
data._train[0]['output']

[[3, 3, 3], [4, 4, 4], [2, 2, 2]]

In [88]:
torch.randn(1, 1, 5).shape

torch.Size([1, 1, 5])

In [89]:
import torch.nn.functional as F

In [90]:
F.pad(sample.flatten(), "constant")

TypeError: pad(): argument 'pad' (position 2) must be tuple of ints, not str

In [91]:
F.pad(sample.flatten().unsqueeze(0), (900 - sample.flatten().shape[0], 900 - sample.flatten().shape[0]), "constant")

tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='mps:0',
       grad_fn=<ConstantPadNdBackward0>)

In [92]:
F.pad(sample.flatten().unsqueeze(0), (900 - sample.flatten().shape[0], 900 - sample.flatten().shape[0]), "constant").shape

torch.Size([1, 1791])

In [97]:
flatten_sample = sample.flatten()

In [98]:
dim = max_shape - flatten_sample.shape[0]

In [99]:
dim

891

In [135]:
def nearest_even_pad(dim: int, input_dim: int):
    if dim % 2 == 1:
        # odd
        dim = dim - 1

    top_padding = dim // 2 # 445
    bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1

    return (top_padding, bot_padding)
print(np.sum(nearest_even_pad(900 - (4*4), 4*4))) # input is 16 
print(np.sum(nearest_even_pad(900 - (17*2), 17*2))) # input is 32
print(np.sum(nearest_even_pad(900 - (3*3), 3*3))) # input is 9

900
900
900


In [136]:
def configure_pad(input_dim: int):
    # input dim is the flattened shape of the vector
    dim = max_flatten_size - input_dim 
    
    if dim % 2 == 1:
        # odd
        dim = dim - 1

    top_padding = dim // 2 # 445
    bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1
    output = (top_padding, bot_padding)
    
    assert np.sum(output) == max_flatten_size, f"Expected padding tuple sum to be {max_flatten_size} but received {np.sum(output)}"
    return output

In [138]:
max_flatten_size = max_shape

In [139]:
max_flatten_size

900

In [140]:
configure_pad(flatten_sample.shape[0])

(445, 455)

In [141]:
pad_dim = configure_pad(flatten_sample.shape[0])

In [144]:
padded = F.pad(flatten_sample, pad_dim, 'constant')

In [150]:
F.pad(flatten_sample, (445, 455-9), 'constant').shape

torch.Size([900])

In [152]:
pad_dim = configure_pad(flatten_sample.shape[0])
pad_dim[0] = pad_dim[0] - flatten_sample.shape[0]
padded = F.pad(flatten_sample, pad_dim, 'constant')
padded.shape

TypeError: 'tuple' object does not support item assignment

In [153]:
pad_dim

(445, 455)

In [154]:
pad_dim[0]

445

In [155]:
def configure_pad(input_dim: int):
    # input dim is the flattened shape of the vector
    dim = max_flatten_size - input_dim 
    
    if dim % 2 == 1:
        # odd
        dim = dim - 1

    top_padding = dim // 2 # 445
    bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1
    output = [top_padding, bot_padding]
    
    assert np.sum(output) == max_flatten_size, f"Expected padding tuple sum to be {max_flatten_size} but received {np.sum(output)}"
    return output

In [157]:
pad_dim = configure_pad(flatten_sample.shape[0])
pad_dim[0] -= flatten_sample.shape[0]
padded = F.pad(flatten_sample, pad_dim, 'constant')
padded.shape

torch.Size([900])

In [158]:
embed2

tensor([[[ 1.4185,  0.2409, -1.7090,  1.4287, -1.4155, -0.4016,  0.8683,
          -1.3092,  1.2262,  1.1998, -0.4632, -0.4990,  0.3592,  1.3017,
          -0.1579, -1.2563,  0.7628, -0.0447,  0.2121, -0.8548, -0.3787,
          -2.3883, -0.8666,  1.5525,  0.2650,  0.0953, -0.4706,  0.2455,
          -0.2126, -0.0498],
         [ 0.1396,  0.6040, -0.1062, -0.6697, -0.4390,  1.4205, -0.4449,
           1.5839, -2.3591, -1.1450, -0.5799, -1.6758, -0.2869,  1.2359,
           1.1060, -0.0571,  0.3278, -0.4090, -1.4151, -0.0621,  0.1338,
           2.6107, -1.3886, -1.0329,  0.8209, -2.4932,  0.6365,  0.2241,
           0.2730,  0.9135],
         [ 0.1396,  0.6040, -0.1062, -0.6697, -0.4390,  1.4205, -0.4449,
           1.5839, -2.3591, -1.1450, -0.5799, -1.6758, -0.2869,  1.2359,
           1.1060, -0.0571,  0.3278, -0.4090, -1.4151, -0.0621,  0.1338,
           2.6107, -1.3886, -1.0329,  0.8209, -2.4932,  0.6365,  0.2241,
           0.2730,  0.9135],
         [ 0.1396,  0.6040, -0.1062, 

In [160]:
emb_layer.shape

torch.Size([30, 30])

In [161]:
emb_layer.flatten().shape

torch.Size([900])

In [162]:
embedding = emb_layer.flatten().unsqueeze(0)
inputs = padded.unsqueeze(0)

In [163]:
embedding.shape, inputs.shape

(torch.Size([1, 900]), torch.Size([1, 900]))

In [164]:
torch.concat(embedding, inputs)

TypeError: concat() received an invalid combination of arguments - got (Tensor, Tensor), but expected one of:
 * (tuple of Tensors tensors, int dim = 0, *, Tensor out = None)
 * (tuple of Tensors tensors, name dim, *, Tensor out = None)


In [165]:
torch.concat([embedding, inputs])

tensor([[2.0120, 0.0581, 2.9206,  ..., 0.0502, 0.0746, 0.8344],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='mps:0', grad_fn=<CatBackward0>)

In [166]:
torch.concat([embedding, inputs]).shape

torch.Size([2, 900])

In [167]:
torch.concat([embedding, inputs]).unsqueeze(0).permute(1, 0, 2)

tensor([[[2.0120, 0.0581, 2.9206,  ..., 0.0502, 0.0746, 0.8344]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
       device='mps:0', grad_fn=<PermuteBackward0>)

In [168]:
torch.concat([embedding, inputs]).unsqueeze(0).permute(1, 0, 2).shape

torch.Size([2, 1, 900])

In [169]:
torch.concat([embedding, inputs]).unsqueeze(0).permute(0, 2, 1).shape

torch.Size([1, 900, 2])

In [171]:
torch.sum(embedding * inputs, dim=0).shape

torch.Size([900])

In [172]:
torch.sum(embedding * inputs, dim=0).tanh()

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [175]:
torch.sum(embedding * inputs, dim=1)

tensor([19.7429], device='mps:0', grad_fn=<SumBackward1>)

In [176]:
(embedding * inputs).tanh()

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [177]:
(embedding * inputs).log_softmax(-1)

tensor([[-10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606, -10.5606,
         -10.5606, -10.5606, -10.5606, -10.5606, -10

In [178]:
torch.sum(embedding*inputs, dim=0)

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.00

In [179]:
torch.sum(embedding*inputs, dim=-1)

tensor([19.7429], device='mps:0', grad_fn=<SumBackward1>)

In [181]:
torch.sum(embedding*inputs, dim=0).shape

torch.Size([900])

In [182]:
torch.matmul(embedding, inputs.T)

tensor([[19.7429]], device='mps:0', grad_fn=<MmBackward0>)

In [183]:
torch.matmul(embedding, inputs)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x900 and 1x900)

In [184]:
torch.matmul(embedding.T, inputs)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='mps:0', grad_fn=<MmBackward0>)

In [185]:
torch.matmul(embedding.T, inputs).shape

torch.Size([900, 900])

In [187]:
[i.sum() for i in torch.matmul(embedding.T, inputs)]

[tensor(30.1803, device='mps:0', grad_fn=<SumBackward0>),
 tensor(0.8708, device='mps:0', grad_fn=<SumBackward0>),
 tensor(43.8094, device='mps:0', grad_fn=<SumBackward0>),
 tensor(30.6184, device='mps:0', grad_fn=<SumBackward0>),
 tensor(30.0544, device='mps:0', grad_fn=<SumBackward0>),
 tensor(2.4186, device='mps:0', grad_fn=<SumBackward0>),
 tensor(11.3097, device='mps:0', grad_fn=<SumBackward0>),
 tensor(25.7100, device='mps:0', grad_fn=<SumBackward0>),
 tensor(22.5518, device='mps:0', grad_fn=<SumBackward0>),
 tensor(21.5922, device='mps:0', grad_fn=<SumBackward0>),
 tensor(3.2183, device='mps:0', grad_fn=<SumBackward0>),
 tensor(3.7355, device='mps:0', grad_fn=<SumBackward0>),
 tensor(1.9356, device='mps:0', grad_fn=<SumBackward0>),
 tensor(25.4162, device='mps:0', grad_fn=<SumBackward0>),
 tensor(0.3739, device='mps:0', grad_fn=<SumBackward0>),
 tensor(23.6728, device='mps:0', grad_fn=<SumBackward0>),
 tensor(8.7287, device='mps:0', grad_fn=<SumBackward0>),
 tensor(0.0300, devic

In [188]:
[i.sum().item() for i in torch.matmul(embedding.T, inputs)]

[30.180273056030273,
 0.8708264827728271,
 43.80940246582031,
 30.61835479736328,
 30.054445266723633,
 2.418639898300171,
 11.309720039367676,
 25.710004806518555,
 22.551828384399414,
 21.592247009277344,
 3.218297004699707,
 3.7354989051818848,
 1.9356446266174316,
 25.416179656982422,
 0.3738875389099121,
 23.67280387878418,
 8.728745460510254,
 0.0299818217754364,
 0.6746166944503784,
 10.960503578186035,
 2.1509735584259033,
 85.5618667602539,
 11.263898849487305,
 36.15477752685547,
 1.0531443357467651,
 0.13612234592437744,
 3.322070360183716,
 0.9037901163101196,
 0.6781933307647705,
 0.037160687148571014,
 0.2923056483268738,
 5.472136974334717,
 0.16920165717601776,
 6.727116107940674,
 2.89017915725708,
 30.2687931060791,
 2.968717575073242,
 37.63050079345703,
 83.47954559326172,
 19.66366958618164,
 5.043592929840088,
 42.12474060058594,
 1.2345175743103027,
 22.911724090576172,
 18.348411560058594,
 0.04884035140275955,
 1.6121513843536377,
 2.5089032649993896,
 30.03882

In [189]:
torch.matmul(embedding, inputs.T)

tensor([[19.7429]], device='mps:0', grad_fn=<MmBackward0>)

In [196]:
torch.matmul(embedding, torch.randn(inputs.shape, device=device).T)

tensor([[1.0833]], device='mps:0', grad_fn=<MmBackward0>)

In [198]:
torch.mean(embedding * torch.randn(inputs.shape, device=device))

tensor(0.2023, device='mps:0', grad_fn=<MeanBackward0>)

In [200]:
torch.mean((embedding * torch.randn(inputs.shape, device=device)).tanh())

tensor(0.0223, device='mps:0', grad_fn=<MeanBackward0>)

In [201]:
torch.mean((embedding * inputs).tanh())

tensor(0.0023, device='mps:0', grad_fn=<MeanBackward0>)

In [202]:
torch.mean(torch.matmul(embedding.T, inputs))

tensor(0.0215, device='mps:0', grad_fn=<MeanBackward0>)

In [203]:
torch.mean(torch.matmul(embedding.T, torch.randn(inputs.shape, device=device)))

tensor(0.0068, device='mps:0', grad_fn=<MeanBackward0>)

In [204]:
random_vec = torch.randn(inputs.shape, device=device)

In [205]:
torch.mean((embedding * random_vec)), torch.mean((embedding * inputs))

(tensor(-0.0342, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(0.0219, device='mps:0', grad_fn=<MeanBackward0>))

In [206]:
torch.mean(torch.matmul(embedding, random_vec.T)), torch.mean(torch.matmul(embedding, inputs.T)), 

(tensor(-30.7569, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(19.7429, device='mps:0', grad_fn=<MeanBackward0>))

In [208]:
torch.matmul(embedding, random_vec.T), torch.matmul(embedding, inputs.T)

(tensor([[-30.7569]], device='mps:0', grad_fn=<MmBackward0>),
 tensor([[19.7429]], device='mps:0', grad_fn=<MmBackward0>))

In [209]:
torch.randn(inputs.shape, device=device).shape

torch.Size([1, 900])

In [212]:
torch.matmul(embedding, random_vec)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x900 and 1x900)

In [213]:
torch.matmul(embedding, random_vec.T)

tensor([[-30.7569]], device='mps:0', grad_fn=<MmBackward0>)

In [214]:
torch.matmul(embedding.T, random_vec)

tensor([[-2.5030e-01,  6.3927e-01,  1.5243e+00,  ...,  3.4940e-01,
         -4.8850e-02, -4.8116e-01],
        [-7.2221e-03,  1.8445e-02,  4.3983e-02,  ...,  1.0082e-02,
         -1.4095e-03, -1.3884e-02],
        [-3.6333e-01,  9.2795e-01,  2.2127e+00,  ...,  5.0719e-01,
         -7.0910e-02, -6.9845e-01],
        ...,
        [-6.2453e-03,  1.5951e-02,  3.8035e-02,  ...,  8.7181e-03,
         -1.2189e-03, -1.2006e-02],
        [-9.2747e-03,  2.3688e-02,  5.6484e-02,  ...,  1.2947e-02,
         -1.8101e-03, -1.7830e-02],
        [-1.0380e-01,  2.6512e-01,  6.3217e-01,  ...,  1.4490e-01,
         -2.0259e-02, -1.9955e-01]], device='mps:0', grad_fn=<MmBackward0>)

In [215]:
torch.matmul(embedding.T, random_vec).mean()

tensor(0.0400, device='mps:0', grad_fn=<MeanBackward0>)

In [216]:
torch.matmul(embedding.T, inputs).mean()

tensor(0.0215, device='mps:0', grad_fn=<MeanBackward0>)

In [217]:
torch.matmul(embedding, torch.randn(inputs.shape, device=device).T)

tensor([[-19.3855]], device='mps:0', grad_fn=<MmBackward0>)

In [219]:
torch.matmul(embedding, inputs.T)

tensor([[19.7429]], device='mps:0', grad_fn=<MmBackward0>)

In [220]:
torch.mean(embedding * inputs.T)

tensor(0.0215, device='mps:0', grad_fn=<MeanBackward0>)

In [221]:
torch.mean(embedding * random_vec.T)

tensor(0.0400, device='mps:0', grad_fn=<MeanBackward0>)

In [222]:
(embedding * random_vec.T).var()

tensor(4.8841, device='mps:0', grad_fn=<VarBackward0>)

In [224]:
(embedding * inputs.T).var()

tensor(0.4108, device='mps:0', grad_fn=<VarBackward0>)

In [225]:
torch.matmul(embedding.T, inputs).var()

tensor(0.4108, device='mps:0', grad_fn=<VarBackward0>)

In [226]:
torch.matmul(embedding.T, random_vec).var()

tensor(4.8841, device='mps:0', grad_fn=<VarBackward0>)

In [227]:
torch.matmul(embedding.T, inputs).mean(), torch.matmul(embedding.T, random_vec).mean()

(tensor(0.0215, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(0.0400, device='mps:0', grad_fn=<MeanBackward0>))

In [228]:
torch.matmul(embedding.T, inputs).tanh()

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='mps:0',
       grad_fn=<TanhBackward0>)

In [229]:
torch.matmul(embedding.T, inputs).tanh().mean()

tensor(0.0024, device='mps:0', grad_fn=<MeanBackward0>)

In [230]:
torch.matmul(embedding.T, random_vec).tanh().mean()

tensor(0.0161, device='mps:0', grad_fn=<MeanBackward0>)

In [231]:
torch.matmul(embedding.T, inputs).tanh().var()

tensor(0.0022, device='mps:0', grad_fn=<VarBackward0>)

In [232]:
torch.matmul(embedding.T, random_vec).tanh().var()

tensor(0.3024, device='mps:0', grad_fn=<VarBackward0>)

In [233]:
torch.matmul(inputs, embedding.T)

tensor([[19.7429]], device='mps:0', grad_fn=<MmBackward0>)

In [234]:
torch.matmul(random_vec, embedding.T)

tensor([[-30.7569]], device='mps:0', grad_fn=<MmBackward0>)

In [235]:
torch.matmul(inputs.T, embedding).mean(), torch.matmul(random_vec.T, embedding).mean()

(tensor(0.0215, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(0.0400, device='mps:0', grad_fn=<MeanBackward0>))

In [236]:
torch.matmul(inputs.T, embedding).var(), torch.matmul(random_vec.T, embedding).var()

(tensor(0.4108, device='mps:0', grad_fn=<VarBackward0>),
 tensor(4.8841, device='mps:0', grad_fn=<VarBackward0>))

In [237]:
torch.matmul(inputs.T, embedding).tanh().var(), torch.matmul(random_vec.T, embedding).tanh().var()

(tensor(0.0022, device='mps:0', grad_fn=<VarBackward0>),
 tensor(0.3024, device='mps:0', grad_fn=<VarBackward0>))

In [238]:
torch.matmul(inputs.T, embedding).tanh().mean(), torch.matmul(random_vec.T, embedding).tanh().mean()

(tensor(0.0024, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(0.0161, device='mps:0', grad_fn=<MeanBackward0>))

In [239]:
def attention(inputs, embedding):
    scores = torch.matmul(embedding.T, inputs)
    attn_scores = torch.tanh(scores / max_flatten_size ** 2)
    return attn_scores

In [240]:
layer = attention(inputs, embedding)

In [241]:
layer.shape

torch.Size([900, 900])

In [242]:
layer.mean(), layer.var()

(tensor(2.6435e-08, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(6.2498e-13, device='mps:0', grad_fn=<VarBackward0>))

In [243]:
layer.mean().item(), layer.var().item()

(2.643469620977612e-08, 6.249804276534099e-13)

In [244]:
def attention(inputs, embedding):
    scores = torch.matmul(embedding.T, inputs)
    attn_scores = torch.tanh(scores) / max_flatten_size ** 2
    return attn_scores

In [245]:
layer = attention(inputs, embedding)

In [246]:
layer.shape

torch.Size([900, 900])

In [247]:
layer.mean(), layer.var()

(tensor(2.9821e-09, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(3.3457e-15, device='mps:0', grad_fn=<VarBackward0>))

In [248]:
import torch

def attention(inputs, embedding):
    # Ensure embedding and inputs are compatible for matrix multiplication
    # Assuming: 
    # - inputs: (batch_size, input_dim)
    # - embedding: (embedding_dim, input_dim)
    
    # Compute the scores using dot product
    scores = torch.matmul(embedding.T, inputs)  # (embedding_dim, batch_size)

    # Apply tanh to the scores
    attn_scores = torch.tanh(scores)
    
    # Scale the scores by the square root of the embedding dimension (common in transformers)
    embedding_dim = embedding.shape[0]  # Assuming embedding is (embedding_dim, input_dim)
    attn_scores = attn_scores / torch.sqrt(torch.tensor(embedding_dim, dtype=torch.float32))
    
    return attn_scores



In [250]:
layer = attention(inputs, embedding)

In [251]:
layer.shape

torch.Size([900, 900])

In [252]:
layer.mean(), layer.var()

(tensor(0.0024, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(0.0022, device='mps:0', grad_fn=<VarBackward0>))

In [253]:
fake_layer = attention(random_vec, embedding)

In [254]:
fake_layer.mean(), fake_layer.var()

(tensor(0.0161, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(0.3024, device='mps:0', grad_fn=<VarBackward0>))

In [255]:
layers = torch.sum(embedding * inputs)

In [256]:
layers = torch.sum(embedding * inputs).to(device)

In [261]:
torch.concat([embedding, inputs]).shape

torch.Size([2, 900])

In [262]:
layers= torch.concat([embedding, inputs]).unsqueeze(0).permute(0, 2, 1)

In [263]:
layers

tensor([[[2.0120, 0.0000],
         [0.0581, 0.0000],
         [2.9206, 0.0000],
         ...,
         [0.0502, 0.0000],
         [0.0746, 0.0000],
         [0.8344, 0.0000]]], device='mps:0', grad_fn=<PermuteBackward0>)

In [264]:
layers.shape

torch.Size([1, 900, 2])

In [265]:
layers.flatten()

tensor([2.0120, 0.0000, 0.0581,  ..., 0.0000, 0.8344, 0.0000], device='mps:0',
       grad_fn=<UnsafeViewBackward0>)

In [266]:
layers.flatten().shape

torch.Size([1800])

In [267]:
torch.sum(embedding, inputs, dim=0)

TypeError: sum() received an invalid combination of arguments - got (Tensor, Tensor, dim=int), but expected one of:
 * (Tensor input, *, torch.dtype dtype = None)
      didn't match because some of the keywords were incorrect: dim
 * (Tensor input, tuple of ints dim, bool keepdim = False, *, torch.dtype dtype = None, Tensor out = None)
 * (Tensor input, tuple of names dim, bool keepdim = False, *, torch.dtype dtype = None, Tensor out = None)


In [268]:
torch.sum([embedding, inputs], dim=0)

TypeError: sum() received an invalid combination of arguments - got (list, dim=int), but expected one of:
 * (Tensor input, *, torch.dtype dtype = None)
 * (Tensor input, tuple of ints dim, bool keepdim = False, *, torch.dtype dtype = None, Tensor out = None)
 * (Tensor input, tuple of names dim, bool keepdim = False, *, torch.dtype dtype = None, Tensor out = None)


In [269]:
torch.sum(torch.concat([embedding, inputs]), dim=0)

tensor([2.0120e+00, 5.8055e-02, 2.9206e+00, 2.0412e+00, 2.0036e+00, 1.6124e-01,
        7.5398e-01, 1.7140e+00, 1.5035e+00, 1.4395e+00, 2.1455e-01, 2.4903e-01,
        1.2904e-01, 1.6944e+00, 2.4926e-02, 1.5782e+00, 5.8192e-01, 1.9988e-03,
        4.4974e-02, 7.3070e-01, 1.4340e-01, 5.7041e+00, 7.5093e-01, 2.4103e+00,
        7.0210e-02, 9.0748e-03, 2.2147e-01, 6.0253e-02, 4.5213e-02, 2.4774e-03,
        1.9487e-02, 3.6481e-01, 1.1280e-02, 4.4847e-01, 1.9268e-01, 2.0179e+00,
        1.9791e-01, 2.5087e+00, 5.5653e+00, 1.3109e+00, 3.3624e-01, 2.8083e+00,
        8.2301e-02, 1.5274e+00, 1.2232e+00, 3.2560e-03, 1.0748e-01, 1.6726e-01,
        2.0026e+00, 3.8582e-03, 1.7910e-02, 6.8159e+00, 1.9281e+00, 1.0670e+00,
        6.7394e-01, 6.2159e+00, 4.0513e-01, 5.0203e-02, 7.4555e-02, 8.3442e-01,
        1.9487e-02, 3.6481e-01, 1.1280e-02, 4.4847e-01, 1.9268e-01, 2.0179e+00,
        1.9791e-01, 2.5087e+00, 5.5653e+00, 1.3109e+00, 3.3624e-01, 2.8083e+00,
        8.2301e-02, 1.5274e+00, 1.2232e+

In [270]:
torch.sum(torch.concat([embedding, inputs]), dim=0).shape

torch.Size([900])

In [271]:
torch.concat([embedding, inputs]).shape

torch.Size([2, 900])

In [272]:
torch.mean(torch.concat([embedding, inputs]), dim=0).shape

torch.Size([900])

In [273]:
torch.mean(torch.concat([embedding, inputs]), dim=-1).shape

torch.Size([2])

In [275]:
torch.sum(torch.concat([embedding, inputs]), dim=0).tanh()

tensor([0.9649, 0.0580, 0.9942, 0.9668, 0.9643, 0.1599, 0.6375, 0.9371, 0.9058,
        0.8936, 0.2113, 0.2440, 0.1283, 0.9347, 0.0249, 0.9183, 0.5241, 0.0020,
        0.0449, 0.6235, 0.1424, 1.0000, 0.6357, 0.9840, 0.0701, 0.0091, 0.2179,
        0.0602, 0.0452, 0.0025, 0.0195, 0.3494, 0.0113, 0.4206, 0.1903, 0.9653,
        0.1954, 0.9868, 1.0000, 0.8645, 0.3241, 0.9928, 0.0821, 0.9100, 0.8406,
        0.0033, 0.1071, 0.1657, 0.9642, 0.0039, 0.0179, 1.0000, 0.9586, 0.7883,
        0.5876, 1.0000, 0.3843, 0.0502, 0.0744, 0.6828, 0.0195, 0.3494, 0.0113,
        0.4206, 0.1903, 0.9653, 0.1954, 0.9868, 1.0000, 0.8645, 0.3241, 0.9928,
        0.0821, 0.9100, 0.8406, 0.0033, 0.1071, 0.1657, 0.9642, 0.0039, 0.0179,
        1.0000, 0.9586, 0.7883, 0.5876, 1.0000, 0.3843, 0.0502, 0.0744, 0.6828,
        0.0195, 0.3494, 0.0113, 0.4206, 0.1903, 0.9653, 0.1954, 0.9868, 1.0000,
        0.8645, 0.3241, 0.9928, 0.0821, 0.9100, 0.8406, 0.0033, 0.1071, 0.1657,
        0.9642, 0.0039, 0.0179, 1.0000, 

In [276]:
torch.sum(torch.concat([embedding, inputs]), dim=0).softmax(-1)

tensor([1.3908e-04, 1.9709e-05, 3.4503e-04, 1.4320e-04, 1.3792e-04, 2.1851e-05,
        3.9527e-05, 1.0324e-04, 8.3635e-05, 7.8452e-05, 2.3048e-05, 2.3856e-05,
        2.1159e-05, 1.0123e-04, 1.9067e-05, 9.0125e-05, 3.3279e-05, 1.8634e-05,
        1.9453e-05, 3.8618e-05, 2.1465e-05, 5.5811e-03, 3.9407e-05, 2.0713e-04,
        1.9950e-05, 1.8767e-05, 2.3208e-05, 1.9752e-05, 1.9457e-05, 1.8643e-05,
        1.8963e-05, 2.6784e-05, 1.8808e-05, 2.9122e-05, 2.2549e-05, 1.3990e-04,
        2.2667e-05, 2.2854e-04, 4.8577e-03, 6.8987e-05, 2.6030e-05, 3.0838e-04,
        2.0192e-05, 8.5666e-05, 6.3196e-05, 1.8658e-05, 2.0707e-05, 2.1983e-05,
        1.3777e-04, 1.8669e-05, 1.8933e-05, 1.6965e-02, 1.2788e-04, 5.4053e-05,
        3.6487e-05, 9.3104e-03, 2.7886e-05, 1.9555e-05, 2.0037e-05, 4.2838e-05,
        1.8963e-05, 2.6784e-05, 1.8808e-05, 2.9122e-05, 2.2549e-05, 1.3990e-04,
        2.2667e-05, 2.2854e-04, 4.8577e-03, 6.8987e-05, 2.6030e-05, 3.0838e-04,
        2.0192e-05, 8.5666e-05, 6.3196e-

In [277]:
torch.sum(torch.concat([embedding, inputs]), dim=0).tanh().mean()

tensor(0.4985, device='mps:0', grad_fn=<MeanBackward0>)

In [278]:
torch.sum(torch.concat([embedding, random_vec]), dim=0).tanh().mean()

tensor(0.4306, device='mps:0', grad_fn=<MeanBackward0>)

In [279]:
torch.sum(torch.concat([embedding, random_vec]), dim=0).tanh().var(), torch.sum(torch.concat([embedding, inputs]), dim=0).tanh().var()

(tensor(0.4013, device='mps:0', grad_fn=<VarBackward0>),
 tensor(0.1569, device='mps:0', grad_fn=<VarBackward0>))

In [280]:
torch.sum(torch.concat([embedding, random_vec]).tanh(), dim=0).mean(), torch.sum(torch.concat([embedding, inputs]).tanh(), dim=0).mean()

(tensor(0.5203, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(0.5007, device='mps:0', grad_fn=<MeanBackward0>))

In [281]:
torch.sum(torch.concat([embedding, random_vec]).tanh(), dim=0).var(), torch.sum(torch.concat([embedding, inputs]).tanh(), dim=0).var()

(tensor(0.5639, device='mps:0', grad_fn=<VarBackward0>),
 tensor(0.1612, device='mps:0', grad_fn=<VarBackward0>))

In [282]:
linear_layer = nn.Linear(900, 900//2, device=device)

In [283]:
linear_layer(torch.sum(torch.concat([embedding, random_vec]).tanh(), dim=0)).shape

torch.Size([450])

In [288]:
real = torch.sum(torch.concat([embedding, inputs]).tanh(), dim=0).unsqueeze(0)
fake = torch.sum(torch.concat([embedding, random_vec]).tanh(), dim=0).unsqueeze(0)



In [289]:
linear_layer(real).shape

torch.Size([1, 450])

In [290]:
linear_layer(real).softmax(-1).mean(), linear_layer(fake).softmax(-1).mean()

(tensor(0.0022, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(0.0022, device='mps:0', grad_fn=<MeanBackward0>))

In [291]:
linear_layer(real).softmax(-1).var(), linear_layer(fake).softmax(-1).var()

(tensor(6.7544e-07, device='mps:0', grad_fn=<VarBackward0>),
 tensor(1.6125e-06, device='mps:0', grad_fn=<VarBackward0>))

In [292]:
linear_layer(real).mean(), linear_layer(fake).mean()

(tensor(-0.0077, device='mps:0', grad_fn=<MeanBackward0>),
 tensor(0.0272, device='mps:0', grad_fn=<MeanBackward0>))

In [293]:
linear_layer(real).var(), linear_layer(fake).var()

(tensor(0.1397, device='mps:0', grad_fn=<VarBackward0>),
 tensor(0.2804, device='mps:0', grad_fn=<VarBackward0>))

In [294]:
from torch.utils.data import DataLoader

In [295]:
from torch.utils.data import DataLoader, TensorDataset
import torch

# Create a random dataset (10 samples, 2 features each)
data = torch.randn(10, 2)
labels = torch.randint(0, 2, (10,))

# Create a TensorDataset
dataset = TensorDataset(data, labels)

# Create a DataLoader with a batch size of 4 and shuffling enabled
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Iterate over the DataLoader
for batch in dataloader:
    inputs, targets = batch
    print('Inputs:', inputs)
    print('Targets:', targets)


Inputs: tensor([[-0.9926, -1.3500],
        [ 0.3220, -0.0501],
        [ 0.2038,  0.1510],
        [ 1.7322, -1.6772]])
Targets: tensor([1, 0, 0, 1])
Inputs: tensor([[-0.4320, -0.5680],
        [ 0.6619,  0.8073],
        [ 0.0689,  0.1086],
        [-2.5812, -2.2332]])
Targets: tensor([0, 0, 0, 1])
Inputs: tensor([[0.8881, 0.2177],
        [1.8765, 0.6289]])
Targets: tensor([1, 1])


In [296]:
for batch in dataloader:
    print(batch)
    break


[tensor([[ 0.2038,  0.1510],
        [ 0.3220, -0.0501],
        [ 0.0689,  0.1086],
        [ 1.7322, -1.6772]]), tensor([0, 0, 0, 1])]


In [297]:
data.shape

torch.Size([10, 2])

In [298]:
labels.shape

torch.Size([10])

In [299]:
train_data

[{'input': [[0, 0, 5], [0, 5, 0], [5, 0, 0]],
  'output': [[3, 3, 3], [4, 4, 4], [2, 2, 2]]},
 {'input': [[0, 0, 5], [0, 0, 5], [0, 0, 5]],
  'output': [[3, 3, 3], [3, 3, 3], [3, 3, 3]]},
 {'input': [[5, 0, 0], [0, 5, 0], [5, 0, 0]],
  'output': [[2, 2, 2], [4, 4, 4], [2, 2, 2]]},
 {'input': [[0, 5, 0], [0, 0, 5], [0, 5, 0]],
  'output': [[4, 4, 4], [3, 3, 3], [4, 4, 4]]},
 {'input': [[3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3],
   [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
   [3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3],
   [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
   [3, 3, 1, 3, 3, 3, 1, 4, 3, 4, 3],
   [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
   [3, 3, 3, 3, 1, 3, 2, 3, 3, 3, 2],
   [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
   [8, 3, 3, 3, 8, 3, 3, 3, 3, 3, 3],
   [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
   [8, 3, 3, 3, 8, 3, 2, 3, 3, 3, 2],
   [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]],
  'output': [[2, 4, 1, 4, 2],
   [8, 3, 3, 3, 8],
   [1, 3, 3, 3, 1],
   [8, 3, 3, 3, 8],
   [2, 4, 1, 4, 2]]},
 {'input': [[1, 1, 1, 1, 1, 1, 1, 1],
   [1, 8

In [300]:
len([i['input'] for i in train_data])

1302

In [301]:
len([i['output'] for i in train_data])

1302

In [302]:
input_data = [i['input'] for i in train_data]

In [303]:
output_data = [i['output'] for i in train_data]

In [304]:
input_data[0]

[[0, 0, 5], [0, 5, 0], [5, 0, 0]]

In [305]:
input_data = [torch.tensor(i['input'], dtype=torch.float, requires_grad=True, device=device) for i in train_data]

In [308]:
output_data = [torch.tensor(i['output'], dtype=torch.float, requires_grad=True, device=device) for i in train_data]

In [309]:
input_data[0]

tensor([[0., 0., 5.],
        [0., 5., 0.],
        [5., 0., 0.]], device='mps:0', requires_grad=True)

In [310]:
input_data[:2]

[tensor([[0., 0., 5.],
         [0., 5., 0.],
         [5., 0., 0.]], device='mps:0', requires_grad=True),
 tensor([[0., 0., 5.],
         [0., 0., 5.],
         [0., 0., 5.]], device='mps:0', requires_grad=True)]

In [311]:
[i.shape for i in input_data[:2]]

[torch.Size([3, 3]), torch.Size([3, 3])]

In [312]:
[{'x_input': i.shape[0], 'y_input': i.shape[-1]} for i in input_data[:2]]

[{'x_input': 3, 'y_input': 3}, {'x_input': 3, 'y_input': 3}]

In [313]:
def vectorise_grid(grid_size: int):
    idx_vec = torch.zeros(max_grid_size, dtype=torch.int, device=device)
    idx_vec[grid_size] = 1
    vec = idx_vec.unsqueeze(0)
    
    assert idx_vec.sum() == 1, f"Expected sum of vector to be 1 but received {idx_vec.sum()}\n Output: {idx_vec}"
    assert list(vec.shape) == [1, 30], f"Expected shape to be [1, 30] but received {vec.shape}"
    return vec 

In [314]:
[{'x_input': vectorise_grid(i.shape[0]), 'y_input': vectorise_grid(i.shape[-1])} for i in input_data[:2]]

[{'x_input': tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0]], device='mps:0', dtype=torch.int32),
  'y_input': tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0]], device='mps:0', dtype=torch.int32)},
 {'x_input': tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0]], device='mps:0', dtype=torch.int32),
  'y_input': tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0]], device='mps:0', dtype=torch.int32)}]

In [315]:
test_dataset = [{'x_input': i.shape[0], 'y_input': i.shape[-1]} for i in input_data[:2]]

In [316]:
from datasets import Dataset

In [319]:
test_grids = Dataset.from_list(test_dataset)

In [322]:
train_data[0]

{'input': [[0, 0, 5], [0, 5, 0], [5, 0, 0]],
 'output': [[3, 3, 3], [4, 4, 4], [2, 2, 2]]}

In [323]:
[{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in train_data[:5]]

[{'input': tensor([[0., 0., 5.],
          [0., 5., 0.],
          [5., 0., 0.]], device='mps:0', requires_grad=True),
  'output': tensor([[3., 3., 3.],
          [4., 4., 4.],
          [2., 2., 2.]], device='mps:0', requires_grad=True)},
 {'input': tensor([[0., 0., 5.],
          [0., 0., 5.],
          [0., 0., 5.]], device='mps:0', requires_grad=True),
  'output': tensor([[3., 3., 3.],
          [3., 3., 3.],
          [3., 3., 3.]], device='mps:0', requires_grad=True)},
 {'input': tensor([[5., 0., 0.],
          [0., 5., 0.],
          [5., 0., 0.]], device='mps:0', requires_grad=True),
  'output': tensor([[2., 2., 2.],
          [4., 4., 4.],
          [2., 2., 2.]], device='mps:0', requires_grad=True)},
 {'input': tensor([[0., 5., 0.],
          [0., 0., 5.],
          [0., 5., 0.]], device='mps:0', requires_grad=True),
  'output': tensor([[4., 4., 4.],
          [3., 3., 3.],
          [4., 4., 4.]], device='mps:0', requires_grad=True)},
 {'input': tensor([[3., 3., 3., 3., 3., 

In [324]:
ss = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in train_data[:5]]

In [325]:
[i.update({'x_shape': i['input'].shape[0], 'y_shape': i['input'].shape[-1]}) for i in ss]

[None, None, None, None, None]

In [326]:
ss[0]

{'input': tensor([[0., 0., 5.],
         [0., 5., 0.],
         [5., 0., 0.]], device='mps:0', requires_grad=True),
 'output': tensor([[3., 3., 3.],
         [4., 4., 4.],
         [2., 2., 2.]], device='mps:0', requires_grad=True),
 'x_shape': 3,
 'y_shape': 3}

In [327]:
train_data[0]['input']

[[0, 0, 5], [0, 5, 0], [5, 0, 0]]

In [329]:
test_grids = Dataset.from_list(ss)

In [330]:
test_grids

Dataset({
    features: ['input', 'output', 'x_shape', 'y_shape'],
    num_rows: 5
})

In [331]:
test_grids['input']

[[[0.0, 0.0, 5.0], [0.0, 5.0, 0.0], [5.0, 0.0, 0.0]],
 [[0.0, 0.0, 5.0], [0.0, 0.0, 5.0], [0.0, 0.0, 5.0]],
 [[5.0, 0.0, 0.0], [0.0, 5.0, 0.0], [5.0, 0.0, 0.0]],
 [[0.0, 5.0, 0.0], [0.0, 0.0, 5.0], [0.0, 5.0, 0.0]],
 [[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 4.0, 3.0],
  [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
  [3.0, 3.0, 3.0, 3.0, 1.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
  [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
  [3.0, 3.0, 1.0, 3.0, 3.0, 3.0, 1.0, 4.0, 3.0, 4.0, 3.0],
  [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
  [3.0, 3.0, 3.0, 3.0, 1.0, 3.0, 2.0, 3.0, 3.0, 3.0, 2.0],
  [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
  [8.0, 3.0, 3.0, 3.0, 8.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
  [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
  [8.0, 3.0, 3.0, 3.0, 8.0, 3.0, 2.0, 3.0, 3.0, 3.0, 2.0],
  [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]]]

In [347]:
test_grids['input'][0]

[[0.0, 0.0, 5.0], [0.0, 5.0, 0.0], [5.0, 0.0, 0.0]]

In [348]:
type(test_grids['input'][0])

list

In [354]:
from torch.utils.data import IterableDataset

class Custom(IterableDataset):
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]
        # update with grid shape sizes
        [i.update({'x_shape': i['input'].shape[0], 'y_shape': i['input'].shape[-1]}) for i in self.data]

        assert set(self.data[0].keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(self.data[0].keys())}'

    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""
        for items in self.data:
            items['x_shape'] = vectorise_grid(items['x_shape'])
            items['y_shape'] = vectorise_grid(items['y_shape'])
            yield items 

custom_iterater = Custom(train_data)

In [355]:
for item in custom_iterater:
    print(item)
    break


{'input': tensor([[0., 0., 5.],
        [0., 5., 0.],
        [5., 0., 0.]], device='mps:0', requires_grad=True), 'output': tensor([[3., 3., 3.],
        [4., 4., 4.],
        [2., 2., 2.]], device='mps:0', requires_grad=True), 'x_shape': tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]], device='mps:0', dtype=torch.int32), 'y_shape': tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]], device='mps:0', dtype=torch.int32)}


In [356]:
for item in custom_iterater:
    print(item.keys())
    break


AssertionError: Expected sum of vector to be 1 but received 2
 Output: tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], device='mps:0', dtype=torch.int32)

In [357]:
from torch.utils.data import IterableDataset

class Custom(IterableDataset):
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]
        # update with grid shape sizes
        #[i.update({'x_shape': i['input'].shape[0], 'y_shape': i['input'].shape[-1]}) for i in self.data]

        #assert set(self.data[0].keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(self.data[0].keys())}'

    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                items['x_shape'] = vectorise_grid(items['input'].shape[0])
                items['y_shape'] = vectorise_grid(items['input'].shape[-1])
            assert set(self.data[0].keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(self.data[0].keys())}'
            yield items

custom_iterater = Custom(train_data)

In [358]:
for item in custom_iterater:
    print(item.keys())
    break


dict_keys(['input', 'output', 'x_shape', 'y_shape'])


In [359]:
for item in custom_iterater:
    print(item.keys())
    break


dict_keys(['input', 'output', 'x_shape', 'y_shape'])


In [360]:
for item in custom_iterater:
    print(item.values())
    break

dict_values([tensor([[0., 0., 5.],
        [0., 5., 0.],
        [5., 0., 0.]], device='mps:0', requires_grad=True), tensor([[3., 3., 3.],
        [4., 4., 4.],
        [2., 2., 2.]], device='mps:0', requires_grad=True), tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]], device='mps:0', dtype=torch.int32), tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]], device='mps:0', dtype=torch.int32)])


In [361]:
from torch.utils.data import IterableDataset

class DataPipe(IterableDataset):
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]

        assert set(self.data[0].keys()) == set(['input', 'output']), f'Expected keys to be [input, output] but received: {self.data[0].keys()}'
        assert isinstance(self.data[0]['input'], torch.Tensor) and isinstance(self.data[0]['output'], torch.Tensor), f"Incorrect dtype for input data: {self.data[0]} where values of {type(self.data[0]['input'])}"
        
    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                items['x_shape'] = vectorise_grid(items['input'].shape[0])
                items['y_shape'] = vectorise_grid(items['input'].shape[-1])
                
            assert set(self.data[0].keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(self.data[0].keys())}'
            
            yield items

class GridDataBasket:
    def __init__(self):
        import glob 
        import json 
        glob_path = f'{directory}/ARC-AGI/data/training/*.json'

        data = []
        for item in glob.glob(glob_path):
            with open(item, 'r') as file:
                data += [json.load(file)]
                
        self.train_data = DataPipe([j for i in data for j in i['train']])
        self.test_data = DataPipe([j for i in data for j in i['test']])
        

In [363]:
grids = GridDataBasket()

In [364]:
for i in grids.train_data:
    print(f"Keys: {i.keys()}")
    break


Keys: dict_keys(['input', 'output', 'x_shape', 'y_shape'])


In [365]:
for i in grids.train_data:
    print(f"Keys: {i.keys()}\nValues: {i.values()}")
    break


Keys: dict_keys(['input', 'output', 'x_shape', 'y_shape'])
Values: dict_values([tensor([[0., 0., 5.],
        [0., 5., 0.],
        [5., 0., 0.]], device='mps:0', requires_grad=True), tensor([[3., 3., 3.],
        [4., 4., 4.],
        [2., 2., 2.]], device='mps:0', requires_grad=True), tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]], device='mps:0', dtype=torch.int32), tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]], device='mps:0', dtype=torch.int32)])


In [368]:

class DataPipe(IterableDataset):
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]

        assert set(self.data[0].keys()) == set(['input', 'output']), f'Expected keys to be [input, output] but received: {self.data[0].keys()}'
        assert isinstance(self.data[0]['input'], torch.Tensor) and isinstance(self.data[0]['output'], torch.Tensor), f"Incorrect dtype for input data: {self.data[0]} where values of {type(self.data[0]['input'])}"
        
    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                # NOTE: transformations to the items are saved in self.data 
                items['input'] = preprocess(items['input'])
                items['output'] = preprocess(items['output'])
                items['x_shape'] = vectorise_grid(items['input'].shape[0])
                items['y_shape'] = vectorise_grid(items['input'].shape[-1])
                
            assert set(self.data[0].keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(self.data[0].keys())}'
            
            yield items

class GridDataBasket:
    def __init__(self):
        import glob 
        import json 
        glob_path = f'{directory}/ARC-AGI/data/training/*.json'

        data = []
        for item in glob.glob(glob_path):
            with open(item, 'r') as file:
                data += [json.load(file)]
                
        self.train_data = DataPipe([j for i in data for j in i['train']])
        self.test_data = DataPipe([j for i in data for j in i['test']])
        
grids = GridDataBasket()

In [370]:
from torch.utils.data import IterableDataset

class DataPipe(IterableDataset):
    @staticmethod
    def configure_pad(input_dim: int):
        # input dim is the flattened shape of the vector
        dim = max_flatten_size - input_dim 
        
        if dim % 2 == 1:
            # odd
            dim = dim - 1

        top_padding = dim // 2 # 445
        bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1
        output = [top_padding, bot_padding]
        
        assert np.sum(output) == max_flatten_size, f"Expected padding tuple sum to be {max_flatten_size} but received {np.sum(output)}"
        return output
    
    @staticmethod
    def preprocess(inputs: torch.Tensor):
        assert len(inputs.shape) == 2, f"Only accept grid n x m but received input of shape: {inputs.shape}"
        
        flatten_input = inputs.flatten()
        flatten_dim = flatten_input.shape[0]
        pad_tuple = DataPipe.configure_pad(flatten_dim)
        pad_tuple[0] -= flatten_dim
        padded = F.pad(flatten_input, pad_tuple, 'constant')
        
        assert tuple(padded.shape) == tuple(max_flatten_size), f"Incorrect padded shape : {padded.shape}"
        return padded
    
    @staticmethod
    def vectorise_grid(grid_size: int):
        idx_vec = torch.zeros(max_grid_size, dtype=torch.int, device=device)
        idx_vec[grid_size] = 1
        vec = idx_vec.unsqueeze(0)
        
        assert idx_vec.sum() == 1, f"Expected sum of vector to be 1 but received {idx_vec.sum()}\n Output: {idx_vec}"
        assert list(vec.shape) == [1, 30], f"Expected shape to be [1, 30] but received {vec.shape}"
        return vec 
    
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]

        assert set(self.data[0].keys()) == set(['input', 'output']), f'Expected keys to be [input, output] but received: {self.data[0].keys()}'
        assert isinstance(self.data[0]['input'], torch.Tensor) and isinstance(self.data[0]['output'], torch.Tensor), f"Incorrect dtype for input data: {self.data[0]} where values of {type(self.data[0]['input'])}"
        
    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                # NOTE: transformations to the items are saved in self.data 
                items['input'] = DataPipe.preprocess(items['input'])
                items['output'] = DataPipe.preprocess(items['output'])
                items['x_shape'] = DataPipe.vectorise_grid(items['input'].shape[0])
                items['y_shape'] = DataPipe.vectorise_grid(items['input'].shape[-1])
                
            assert set(items.keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(items.keys())}'
            
            yield items


In [371]:
grids = GridDataBasket()

In [372]:
for item in grids.train_data:
    print(item)
    break



TypeError: 'int' object is not iterable

In [373]:
class DataPipe(IterableDataset):
    @staticmethod
    def configure_pad(input_dim: int):
        # input dim is the flattened shape of the vector
        dim = max_flatten_size - input_dim 
        
        if dim % 2 == 1:
            # odd
            dim = dim - 1

        top_padding = dim // 2 # 445
        bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1
        output = [top_padding, bot_padding]
        
        assert np.sum(output) == max_flatten_size, f"Expected padding tuple sum to be {max_flatten_size} but received {np.sum(output)}"
        return output
    
    @staticmethod
    def preprocess(inputs: torch.Tensor):
        assert len(inputs.shape) == 2, f"Only accept grid n x m but received input of shape: {inputs.shape}"
        
        flatten_input = inputs.flatten()
        flatten_dim = flatten_input.shape[0]
        pad_tuple = DataPipe.configure_pad(flatten_dim)
        pad_tuple[0] -= flatten_dim
        padded = F.pad(flatten_input, pad_tuple, 'constant')
        
        assert set(padded.shape) == set([1, max_flatten_size]), f"Incorrect padded shape : {padded.shape}"
        return padded
    
    @staticmethod
    def vectorise_grid(grid_size: int):
        idx_vec = torch.zeros(max_grid_size, dtype=torch.int, device=device)
        idx_vec[grid_size] = 1
        vec = idx_vec.unsqueeze(0)
        
        assert idx_vec.sum() == 1, f"Expected sum of vector to be 1 but received {idx_vec.sum()}\n Output: {idx_vec}"
        assert list(vec.shape) == [1, 30], f"Expected shape to be [1, 30] but received {vec.shape}"
        return vec 
    
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]

        assert set(self.data[0].keys()) == set(['input', 'output']), f'Expected keys to be [input, output] but received: {self.data[0].keys()}'
        assert isinstance(self.data[0]['input'], torch.Tensor) and isinstance(self.data[0]['output'], torch.Tensor), f"Incorrect dtype for input data: {self.data[0]} where values of {type(self.data[0]['input'])}"
        
    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                # NOTE: transformations to the items are saved in self.data 
                items['input'] = DataPipe.preprocess(items['input'])
                items['output'] = DataPipe.preprocess(items['output'])
                items['x_shape'] = DataPipe.vectorise_grid(items['input'].shape[0])
                items['y_shape'] = DataPipe.vectorise_grid(items['input'].shape[-1])
                
            assert set(items.keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(items.keys())}'
            
            yield items

class GridDataBasket:
    def __init__(self):
        import glob 
        import json 
        glob_path = f'{directory}/ARC-AGI/data/training/*.json'

        data = []
        for item in glob.glob(glob_path):
            with open(item, 'r') as file:
                data += [json.load(file)]
                
        self.train_data = DataPipe([j for i in data for j in i['train']])
        self.test_data = DataPipe([j for i in data for j in i['test']])
        
grids = GridDataBasket()


In [374]:
for i in grids.train_data:
    print(i)
    break


AssertionError: Incorrect padded shape : torch.Size([900])

In [375]:

class DataPipe(IterableDataset):
    @staticmethod
    def configure_pad(input_dim: int):
        # input dim is the flattened shape of the vector
        dim = max_flatten_size - input_dim 
        
        if dim % 2 == 1:
            # odd
            dim = dim - 1

        top_padding = dim // 2 # 445
        bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1
        output = [top_padding, bot_padding]
        
        assert np.sum(output) == max_flatten_size, f"Expected padding tuple sum to be {max_flatten_size} but received {np.sum(output)}"
        return output
    
    @staticmethod
    def preprocess(inputs: torch.Tensor):
        assert len(inputs.shape) == 2, f"Only accept grid n x m but received input of shape: {inputs.shape}"
        
        flatten_input = inputs.flatten()
        flatten_dim = flatten_input.shape[0]
        pad_tuple = DataPipe.configure_pad(flatten_dim)
        pad_tuple[0] -= flatten_dim
        padded = F.pad(flatten_input, pad_tuple, 'constant')
    
        assert padded.shape == max_flatten_size, f"Incorrect padded shape : {padded.shape}"
        return padded
    
    @staticmethod
    def vectorise_grid(grid_size: int):
        idx_vec = torch.zeros(max_grid_size, dtype=torch.int, device=device)
        idx_vec[grid_size] = 1
        vec = idx_vec.unsqueeze(0)
        
        assert idx_vec.sum() == 1, f"Expected sum of vector to be 1 but received {idx_vec.sum()}\n Output: {idx_vec}"
        assert list(vec.shape) == [1, 30], f"Expected shape to be [1, 30] but received {vec.shape}"
        return vec 
    
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]

        assert set(self.data[0].keys()) == set(['input', 'output']), f'Expected keys to be [input, output] but received: {self.data[0].keys()}'
        assert isinstance(self.data[0]['input'], torch.Tensor) and isinstance(self.data[0]['output'], torch.Tensor), f"Incorrect dtype for input data: {self.data[0]} where values of {type(self.data[0]['input'])}"
        
    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                # NOTE: transformations to the items are saved in self.data 
                items['input'] = DataPipe.preprocess(items['input'])
                items['output'] = DataPipe.preprocess(items['output'])
                items['x_shape'] = DataPipe.vectorise_grid(items['input'].shape[0])
                items['y_shape'] = DataPipe.vectorise_grid(items['input'].shape[-1])
                
            assert set(items.keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(items.keys())}'
            
            yield items

class GridDataBasket:
    def __init__(self):
        import glob 
        import json 
        glob_path = f'{directory}/ARC-AGI/data/training/*.json'

        data = []
        for item in glob.glob(glob_path):
            with open(item, 'r') as file:
                data += [json.load(file)]
                
        self.train_data = DataPipe([j for i in data for j in i['train']])
        self.test_data = DataPipe([j for i in data for j in i['test']])
grids = GridDataBasket()
for i in grids.train_data:
    print(i)
    break


AssertionError: Incorrect padded shape : torch.Size([900])

In [376]:

class DataPipe(IterableDataset):
    @staticmethod
    def configure_pad(input_dim: int):
        # input dim is the flattened shape of the vector
        dim = max_flatten_size - input_dim 
        
        if dim % 2 == 1:
            # odd
            dim = dim - 1

        top_padding = dim // 2 # 445
        bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1
        output = [top_padding, bot_padding]
        
        assert np.sum(output) == max_flatten_size, f"Expected padding tuple sum to be {max_flatten_size} but received {np.sum(output)}"
        return output
    
    @staticmethod
    def preprocess(inputs: torch.Tensor):
        assert len(inputs.shape) == 2, f"Only accept grid n x m but received input of shape: {inputs.shape}"
        
        flatten_input = inputs.flatten()
        flatten_dim = flatten_input.shape[0]
        pad_tuple = DataPipe.configure_pad(flatten_dim)
        pad_tuple[0] -= flatten_dim
        padded = F.pad(flatten_input, pad_tuple, 'constant')
    
        assert padded.shape[0] == max_flatten_size, f"Incorrect padded shape : {padded.shape}"
        return padded
    
    @staticmethod
    def vectorise_grid(grid_size: int):
        idx_vec = torch.zeros(max_grid_size, dtype=torch.int, device=device)
        idx_vec[grid_size] = 1
        vec = idx_vec.unsqueeze(0)
        
        assert idx_vec.sum() == 1, f"Expected sum of vector to be 1 but received {idx_vec.sum()}\n Output: {idx_vec}"
        assert list(vec.shape) == [1, 30], f"Expected shape to be [1, 30] but received {vec.shape}"
        return vec 
    
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]

        assert set(self.data[0].keys()) == set(['input', 'output']), f'Expected keys to be [input, output] but received: {self.data[0].keys()}'
        assert isinstance(self.data[0]['input'], torch.Tensor) and isinstance(self.data[0]['output'], torch.Tensor), f"Incorrect dtype for input data: {self.data[0]} where values of {type(self.data[0]['input'])}"
        
    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                # NOTE: transformations to the items are saved in self.data 
                items['input'] = DataPipe.preprocess(items['input'])
                items['output'] = DataPipe.preprocess(items['output'])
                items['x_shape'] = DataPipe.vectorise_grid(items['input'].shape[0])
                items['y_shape'] = DataPipe.vectorise_grid(items['input'].shape[-1])
                
            assert set(items.keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(items.keys())}'
            
            yield items

class GridDataBasket:
    def __init__(self):
        import glob 
        import json 
        glob_path = f'{directory}/ARC-AGI/data/training/*.json'

        data = []
        for item in glob.glob(glob_path):
            with open(item, 'r') as file:
                data += [json.load(file)]
                
        self.train_data = DataPipe([j for i in data for j in i['train']])
        self.test_data = DataPipe([j for i in data for j in i['test']])

grids = GridDataBasket()
for i in grids.train_data:
    print(i)
    break


IndexError: index 900 is out of bounds for dimension 0 with size 30

In [377]:

class DataPipe(IterableDataset):
    @staticmethod
    def configure_pad(input_dim: int):
        # input dim is the flattened shape of the vector
        dim = max_flatten_size - input_dim 
        
        if dim % 2 == 1:
            # odd
            dim = dim - 1

        top_padding = dim // 2 # 445
        bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1
        output = [top_padding, bot_padding]
        
        assert np.sum(output) == max_flatten_size, f"Expected padding tuple sum to be {max_flatten_size} but received {np.sum(output)}"
        return output
    
    @staticmethod
    def preprocess(inputs: torch.Tensor):
        assert len(inputs.shape) == 2, f"Only accept grid n x m but received input of shape: {inputs.shape}"
        
        flatten_input = inputs.flatten()
        flatten_dim = flatten_input.shape[0]
        pad_tuple = DataPipe.configure_pad(flatten_dim)
        pad_tuple[0] -= flatten_dim
        padded = F.pad(flatten_input, pad_tuple, 'constant')
    
        assert padded.shape[0].items() == max_flatten_size, f"Incorrect padded shape : {padded.shape}"
        return padded
    
    @staticmethod
    def vectorise_grid(grid_size: int):
        idx_vec = torch.zeros(max_grid_size, dtype=torch.int, device=device)
        idx_vec[grid_size] = 1
        vec = idx_vec.unsqueeze(0)
        
        assert idx_vec.sum() == 1, f"Expected sum of vector to be 1 but received {idx_vec.sum()}\n Output: {idx_vec}"
        assert list(vec.shape) == [1, 30], f"Expected shape to be [1, 30] but received {vec.shape}"
        return vec 
    
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]

        assert set(self.data[0].keys()) == set(['input', 'output']), f'Expected keys to be [input, output] but received: {self.data[0].keys()}'
        assert isinstance(self.data[0]['input'], torch.Tensor) and isinstance(self.data[0]['output'], torch.Tensor), f"Incorrect dtype for input data: {self.data[0]} where values of {type(self.data[0]['input'])}"
        
    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                # NOTE: transformations to the items are saved in self.data 
                items['input'] = DataPipe.preprocess(items['input'])
                items['output'] = DataPipe.preprocess(items['output'])
                items['x_shape'] = DataPipe.vectorise_grid(items['input'].shape[0])
                items['y_shape'] = DataPipe.vectorise_grid(items['input'].shape[-1])
                
            assert set(items.keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(items.keys())}'
            
            yield items

class GridDataBasket:
    def __init__(self):
        import glob 
        import json 
        glob_path = f'{directory}/ARC-AGI/data/training/*.json'

        data = []
        for item in glob.glob(glob_path):
            with open(item, 'r') as file:
                data += [json.load(file)]
                
        self.train_data = DataPipe([j for i in data for j in i['train']])
        self.test_data = DataPipe([j for i in data for j in i['test']])

grids = GridDataBasket()
for i in grids.train_data:
    print(i)
    break


AttributeError: 'int' object has no attribute 'items'

In [378]:

class DataPipe(IterableDataset):
    @staticmethod
    def configure_pad(input_dim: int):
        # input dim is the flattened shape of the vector
        dim = max_flatten_size - input_dim 
        
        if dim % 2 == 1:
            # odd
            dim = dim - 1

        top_padding = dim // 2 # 445
        bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1
        output = [top_padding, bot_padding]
        
        assert np.sum(output) == max_flatten_size, f"Expected padding tuple sum to be {max_flatten_size} but received {np.sum(output)}"
        return output
    
    @staticmethod
    def preprocess(inputs: torch.Tensor):
        assert len(inputs.shape) == 2, f"Only accept grid n x m but received input of shape: {inputs.shape}"
        
        flatten_input = inputs.flatten()
        flatten_dim = flatten_input.shape[0]
        pad_tuple = DataPipe.configure_pad(flatten_dim)
        pad_tuple[0] -= flatten_dim
        padded = F.pad(flatten_input, pad_tuple, 'constant')
    
        assert padded.shape[0].item() == max_flatten_size, f"Incorrect padded shape : {padded.shape}"
        return padded
    
    @staticmethod
    def vectorise_grid(grid_size: int):
        idx_vec = torch.zeros(max_grid_size, dtype=torch.int, device=device)
        idx_vec[grid_size] = 1
        vec = idx_vec.unsqueeze(0)
        
        assert idx_vec.sum() == 1, f"Expected sum of vector to be 1 but received {idx_vec.sum()}\n Output: {idx_vec}"
        assert list(vec.shape) == [1, 30], f"Expected shape to be [1, 30] but received {vec.shape}"
        return vec 
    
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]

        assert set(self.data[0].keys()) == set(['input', 'output']), f'Expected keys to be [input, output] but received: {self.data[0].keys()}'
        assert isinstance(self.data[0]['input'], torch.Tensor) and isinstance(self.data[0]['output'], torch.Tensor), f"Incorrect dtype for input data: {self.data[0]} where values of {type(self.data[0]['input'])}"
        
    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                # NOTE: transformations to the items are saved in self.data 
                items['input'] = DataPipe.preprocess(items['input'])
                items['output'] = DataPipe.preprocess(items['output'])
                items['x_shape'] = DataPipe.vectorise_grid(items['input'].shape[0])
                items['y_shape'] = DataPipe.vectorise_grid(items['input'].shape[-1])
                
            assert set(items.keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(items.keys())}'
            
            yield items

class GridDataBasket:
    def __init__(self):
        import glob 
        import json 
        glob_path = f'{directory}/ARC-AGI/data/training/*.json'

        data = []
        for item in glob.glob(glob_path):
            with open(item, 'r') as file:
                data += [json.load(file)]
                
        self.train_data = DataPipe([j for i in data for j in i['train']])
        self.test_data = DataPipe([j for i in data for j in i['test']])

grids = GridDataBasket()
for i in grids.train_data:
    print(i)
    break


AttributeError: 'int' object has no attribute 'item'

In [379]:

class DataPipe(IterableDataset):
    @staticmethod
    def configure_pad(input_dim: int):
        # input dim is the flattened shape of the vector
        dim = max_flatten_size - input_dim 
        
        if dim % 2 == 1:
            # odd
            dim = dim - 1

        top_padding = dim // 2 # 445
        bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1
        output = [top_padding, bot_padding]
        
        assert np.sum(output) == max_flatten_size, f"Expected padding tuple sum to be {max_flatten_size} but received {np.sum(output)}"
        return output
    
    @staticmethod
    def preprocess(inputs: torch.Tensor):
        assert len(inputs.shape) == 2, f"Only accept grid n x m but received input of shape: {inputs.shape}"
        
        flatten_input = inputs.flatten()
        flatten_dim = flatten_input.shape[0]
        pad_tuple = DataPipe.configure_pad(flatten_dim)
        pad_tuple[0] -= flatten_dim
        padded = F.pad(flatten_input, pad_tuple, 'constant')
    
        assert padded.shape == torch.Size([max_flatten_size]), f"Incorrect padded shape : {padded.shape}"
        return padded
    
    @staticmethod
    def vectorise_grid(grid_size: int):
        idx_vec = torch.zeros(max_grid_size, dtype=torch.int, device=device)
        idx_vec[grid_size] = 1
        vec = idx_vec.unsqueeze(0)
        
        assert idx_vec.sum() == 1, f"Expected sum of vector to be 1 but received {idx_vec.sum()}\n Output: {idx_vec}"
        assert list(vec.shape) == [1, 30], f"Expected shape to be [1, 30] but received {vec.shape}"
        return vec 
    
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]

        assert set(self.data[0].keys()) == set(['input', 'output']), f'Expected keys to be [input, output] but received: {self.data[0].keys()}'
        assert isinstance(self.data[0]['input'], torch.Tensor) and isinstance(self.data[0]['output'], torch.Tensor), f"Incorrect dtype for input data: {self.data[0]} where values of {type(self.data[0]['input'])}"
        
    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                # NOTE: transformations to the items are saved in self.data 
                items['input'] = DataPipe.preprocess(items['input'])
                items['output'] = DataPipe.preprocess(items['output'])
                items['x_shape'] = DataPipe.vectorise_grid(items['input'].shape[0])
                items['y_shape'] = DataPipe.vectorise_grid(items['input'].shape[-1])
                
            assert set(items.keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(items.keys())}'
            
            yield items

class GridDataBasket:
    def __init__(self):
        import glob 
        import json 
        glob_path = f'{directory}/ARC-AGI/data/training/*.json'

        data = []
        for item in glob.glob(glob_path):
            with open(item, 'r') as file:
                data += [json.load(file)]
                
        self.train_data = DataPipe([j for i in data for j in i['train']])
        self.test_data = DataPipe([j for i in data for j in i['test']])

grids = GridDataBasket()
for i in grids.train_data:
    print(i)
    break


IndexError: index 900 is out of bounds for dimension 0 with size 30

In [380]:

class DataPipe(IterableDataset):
    @staticmethod
    def configure_pad(input_dim: int):
        # input dim is the flattened shape of the vector
        dim = max_flatten_size - input_dim 
        
        if dim % 2 == 1:
            # odd
            dim = dim - 1

        top_padding = dim // 2 # 445
        bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1
        output = [top_padding, bot_padding]
        
        assert np.sum(output) == max_flatten_size, f"Expected padding tuple sum to be {max_flatten_size} but received {np.sum(output)}"
        print(f'Returning pad shape: {output}')
        return output
    
    @staticmethod
    def preprocess(inputs: torch.Tensor):
        assert len(inputs.shape) == 2, f"Only accept grid n x m but received input of shape: {inputs.shape}"
        
        flatten_input = inputs.flatten()
        flatten_dim = flatten_input.shape[0]
        pad_tuple = DataPipe.configure_pad(flatten_dim)
        pad_tuple[0] -= flatten_dim
        padded = F.pad(flatten_input, pad_tuple, 'constant')
        print(f'Padded vec: {padded}\nShape: {padded.shape}')
        assert padded.shape[0] == max_flatten_size, f"Incorrect padded shape : {padded.shape}"
        return padded
    
    @staticmethod
    def vectorise_grid(grid_size: int):
        idx_vec = torch.zeros(max_grid_size, dtype=torch.int, device=device)
        idx_vec[grid_size] = 1
        vec = idx_vec.unsqueeze(0)
        
        assert idx_vec.sum() == 1, f"Expected sum of vector to be 1 but received {idx_vec.sum()}\n Output: {idx_vec}"
        assert list(vec.shape) == [1, 30], f"Expected shape to be [1, 30] but received {vec.shape}"
        return vec 
    
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]

        assert set(self.data[0].keys()) == set(['input', 'output']), f'Expected keys to be [input, output] but received: {self.data[0].keys()}'
        assert isinstance(self.data[0]['input'], torch.Tensor) and isinstance(self.data[0]['output'], torch.Tensor), f"Incorrect dtype for input data: {self.data[0]} where values of {type(self.data[0]['input'])}"
        
    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                # NOTE: transformations to the items are saved in self.data 
                items['input'] = DataPipe.preprocess(items['input'])
                items['output'] = DataPipe.preprocess(items['output'])
                items['x_shape'] = DataPipe.vectorise_grid(items['input'].shape[0])
                items['y_shape'] = DataPipe.vectorise_grid(items['input'].shape[-1])
                
            assert set(items.keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(items.keys())}'
            
            yield items

class GridDataBasket:
    def __init__(self):
        import glob 
        import json 
        glob_path = f'{directory}/ARC-AGI/data/training/*.json'

        data = []
        for item in glob.glob(glob_path):
            with open(item, 'r') as file:
                data += [json.load(file)]
                
        self.train_data = DataPipe([j for i in data for j in i['train']])
        self.test_data = DataPipe([j for i in data for j in i['test']])

grids = GridDataBasket()
for i in grids.train_data:
    print(i)
    break


Returning pad shape: [445, 455]
Padded vec: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

IndexError: index 900 is out of bounds for dimension 0 with size 30

In [381]:

class DataPipe(IterableDataset):
    @staticmethod
    def configure_pad(input_dim: int):
        # input dim is the flattened shape of the vector
        dim = max_flatten_size - input_dim 
        
        if dim % 2 == 1:
            # odd
            dim = dim - 1

        top_padding = dim // 2 # 445
        bot_padding = top_padding + input_dim if input_dim % 2 == 0 else top_padding + input_dim + 1
        output = [top_padding, bot_padding]
        
        assert np.sum(output) == max_flatten_size, f"Expected padding tuple sum to be {max_flatten_size} but received {np.sum(output)}"
        print(f'Returning pad shape: {output}')
        return output
    
    @staticmethod
    def preprocess(inputs: torch.Tensor):
        assert len(inputs.shape) == 2, f"Only accept grid n x m but received input of shape: {inputs.shape}"
        
        flatten_input = inputs.flatten()
        flatten_dim = flatten_input.shape[0]
        pad_tuple = DataPipe.configure_pad(flatten_dim)
        pad_tuple[0] -= flatten_dim
        padded = F.pad(flatten_input, pad_tuple, 'constant')
        print(f'Padded vec: {padded}\nShape: {padded.shape}')
        assert padded.shape[0] == max_flatten_size, f"Incorrect padded shape : {padded.shape}"
        return padded
    
    @staticmethod
    def vectorise_grid(grid_size: int):
        idx_vec = torch.zeros(max_grid_size, dtype=torch.int, device=device)
        idx_vec[grid_size] = 1
        vec = idx_vec.unsqueeze(0)
        
        assert idx_vec.sum() == 1, f"Expected sum of vector to be 1 but received {idx_vec.sum()}\n Output: {idx_vec}"
        assert list(vec.shape) == [1, 30], f"Expected shape to be [1, 30] but received {vec.shape}"
        return vec 
    
    def __init__(self, dataset: list):
        """
        Args:
            inputs: A tensor containing the input data (e.g., shape [N, M]).
            outputs: A tensor containing the corresponding labels or outputs.
        """
        # convert to tensor arrays 
        self.data = [{keys: torch.tensor(vals, dtype=torch.float, device=device, requires_grad=True) for keys, vals in i.items()} for i in dataset]

        assert set(self.data[0].keys()) == set(['input', 'output']), f'Expected keys to be [input, output] but received: {self.data[0].keys()}'
        assert isinstance(self.data[0]['input'], torch.Tensor) and isinstance(self.data[0]['output'], torch.Tensor), f"Incorrect dtype for input data: {self.data[0]} where values of {type(self.data[0]['input'])}"
        
    def __iter__(self):
        """Retrieve a sample from the dataset at the given index."""

        for items in self.data:
            if 'x_shape' not in items or 'y_shape' not in items:
                # NOTE: transformations to the items are saved in self.data 
                items['x_shape'] = DataPipe.vectorise_grid(items['input'].shape[0])
                items['y_shape'] = DataPipe.vectorise_grid(items['input'].shape[-1])
                items['input'] = DataPipe.preprocess(items['input'])
                items['output'] = DataPipe.preprocess(items['output'])

            assert set(items.keys()) == set(['x_shape', 'y_shape', 'input', 'output']), f'Incorrect keys in preprocessed data. Listed keys in the data: {list(items.keys())}'
            
            yield items

class GridDataBasket:
    def __init__(self):
        import glob 
        import json 
        glob_path = f'{directory}/ARC-AGI/data/training/*.json'

        data = []
        for item in glob.glob(glob_path):
            with open(item, 'r') as file:
                data += [json.load(file)]
                
        self.train_data = DataPipe([j for i in data for j in i['train']])
        self.test_data = DataPipe([j for i in data for j in i['test']])

grids = GridDataBasket()
for i in grids.train_data:
    print(i)
    break


Returning pad shape: [445, 455]
Padded vec: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [382]:
list(grids.train_data)

Returning pad shape: [445, 455]
Padded vec: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

IndexError: index 30 is out of bounds for dimension 0 with size 30

In [383]:
DataLoader(grids.train_data, batch_size=2)

In [384]:
grid_loader = DataLoader(grids.train_data, batch_size=2, shuffle=True)

ValueError: DataLoader with IterableDataset: expected unspecified shuffle option, but got shuffle=True